<a href="https://colab.research.google.com/github/sursingh/bookkeeper/blob/master/sp500/pairings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
import os
from google.colab import userdata
#from google.cloud import storage

os.environ['GOOGLE_CLOUD_PROJECT'] = userdata.get('GCP_PROJECT')

bucket = "glenpro-stock-data"
prefix = "raw/yf/sp500/"

In [69]:
from scipy.cluster.hierarchy import dendrogram, linkage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [3]:
!mkdir -p data/yf
!gsutil -m cp gs://{bucket}/{prefix}yf-sp500*.tar.gz data/
!tar -xzf data/yf-sp500*.tar.gz -C .

Copying gs://glenpro-stock-data/raw/yf/sp500/yf-sp500-20240826-033158.tar.gz...
\ [1/1 files][ 92.2 MiB/ 92.2 MiB] 100% Done                                    
Operation completed over 1 objects/92.2 MiB.                                     


In [79]:
# get SP500 constituents from wikipedia

df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500Info = df.set_index("Symbol")

# Get sp500 company weights
df = pd.read_html(requests.get('https://www.slickcharts.com/sp500',
                      headers={'User-agent': 'Mozilla/5.0'}).text)[0]
df.set_index("Symbol", inplace=True)
weights = df["Portfolio%"].str.replace("%", "").astype(float)
sp500Info["Weight"] = weights

sp500Info.head()

<ipython-input-79-4ffe32e8d486>:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(requests.get('https://www.slickcharts.com/sp500',


,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Weight
Symbol,,,,,,,,
MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,0.15
AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,0.02
ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,0.42
ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),0.74
ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,0.44


In [11]:
import pandas as pd
import numpy as np


def load_ticker(ticker):
  df = pd.read_csv(f"data/yf/{ticker}.csv")
  df["symbol"] = ticker
  df["Date"] = pd.to_datetime(df["Date"])
  df.set_index("Date", inplace=True)
  return df

def calc_returns(ticker):
  df = load_ticker(ticker)
  lc = np.log(df["Adj Close"])
  return lc.diff()

def getTickers():
  for ticker in os.listdir("data/yf"):
    if ticker.endswith(".csv"):
      yield ticker[:-4]

tickers = list(getTickers())

In [12]:
returns = pd.DataFrame(
    {
        ticker: calc_returns(ticker)
        for ticker in tickers
    }
)
returns.head()

,NTAP,DOW,LEN,RMD,CL,EMR,MS,MOS,PNR,AMP,...,LVS,HPQ,MRO,EQR,OKE,MCD,SNA,BIIB,COP,GEN
Date,,,,,,,,,,,,,,,,,,,,,
1962-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.010777,-0.006135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.021899,-0.003081,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.022381,-0.006193,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.014982,0.006193,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
returns.fillna(0, inplace=True)

In [30]:
r18 = returns['2018':]

In [31]:
r18

,NTAP,DOW,LEN,RMD,CL,EMR,MS,MOS,PNR,AMP,...,LVS,HPQ,MRO,EQR,OKE,MCD,SNA,BIIB,COP,GEN
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,0.004509,0.000000,0.024986,0.009870,-0.004117,0.014954,-0.005351,0.037856,-0.005538,0.002593,...,-0.005628,0.014648,0.026233,-0.004086,0.022202,0.006371,0.014751,0.047808,0.007803,0.029842
2018-01-03,0.026635,0.000000,0.036197,0.010352,-0.003867,0.009147,0.002679,-0.009047,0.008083,-0.004956,...,-0.011352,0.002342,0.016548,0.005183,0.016154,-0.004224,-0.009657,0.016854,0.018270,0.000000
2018-01-04,0.016394,0.000000,-0.004174,0.001388,0.006392,0.001400,0.015172,0.015405,0.024417,0.003660,...,-0.002491,0.007459,0.025148,-0.017381,0.011813,0.006991,0.005578,0.000412,0.011471,0.002073
2018-01-05,0.017830,0.000000,0.009366,0.013201,0.001194,0.013063,-0.000377,0.001490,0.001515,0.003412,...,-0.003381,0.010167,-0.012774,-0.000797,-0.002494,0.002013,0.012859,0.007326,-0.001932,0.018468
2018-01-08,0.013502,0.000000,-0.005192,0.005686,0.001457,-0.003180,-0.004151,-0.021450,-0.000964,0.010109,...,0.001177,0.004129,0.004462,0.006677,0.010293,-0.000690,0.016782,-0.038211,0.010145,0.002031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-19,0.009153,-0.004834,0.021370,0.015467,-0.001374,0.007249,0.003051,0.008263,0.004368,0.009115,...,0.011820,-0.037162,0.007782,0.002517,0.010690,0.032015,0.010984,0.020345,0.005024,0.013235
2024-08-20,-0.002357,-0.017487,-0.008413,-0.035871,0.011810,-0.006958,-0.009579,-0.024997,-0.008637,-0.003014,...,0.002933,0.006051,-0.021008,0.005433,-0.020688,-0.006699,-0.007273,-0.002770,-0.017059,-0.006694
2024-08-21,0.015257,0.004353,0.028610,0.005140,0.003970,0.008690,-0.000893,0.023923,0.007812,-0.000968,...,0.007780,0.013128,-0.001808,0.004574,0.005069,0.014183,0.012882,0.003352,-0.002096,0.017235


In [32]:
corr = r18.corr()

# get entries in order of correlation
corr_entries = corr.stack()
corr_entries.sort_values(inplace=True)
corr_entries.head()

,,0
SW,K,-0.096445
K,SW,-0.096445
CLX,FANG,-0.095650
FANG,CLX,-0.095650
K,UBER,-0.090996


In [44]:
corr_entries = corr_entries[corr_entries!=1]

In [45]:
corr_entries

,,0
SW,K,-0.096445
K,SW,-0.096445
CLX,FANG,-0.095650
FANG,CLX,-0.095650
K,UBER,-0.090996
...,...,...
NWSA,NWS,0.977964
FOXA,FOX,0.986102
FOX,FOXA,0.986102
GOOG,GOOGL,0.995323


In [47]:
scored = corr_entries.reset_index()

In [51]:
scored.columns = ["ticker1", "ticker2", "correlation"]
scored = scored[scored.ticker1 > scored.ticker2]


In [80]:
for _, row in scored.tail(20).iterrows():
  t1 = sp500Info.loc[row.ticker1]
  t2 = sp500Info.loc[row.ticker2]
  print(f"{row.ticker1} {row.ticker2} {row.correlation}")
  print(f"\t{t1.Security} {t1['GICS Sector']} {t1['GICS Sub-Industry']} {t1.Weight}")
  print(f"\t{t2.Security} {t2['GICS Sector']} {t2['GICS Sub-Industry']} {t2.Weight}")

LEN DHI 0.9020922819336682
	Lennar Consumer Discretionary Homebuilding 0.09
	DR Horton Consumer Discretionary Homebuilding 0.12
LNT CMS 0.9029584682105047
	Alliant Energy Utilities Electric Utilities 0.03
	CMS Energy Utilities Multi-Utilities 0.04
KEY FITB 0.9032084172400648
	KeyCorp Financials Regional Banks 0.03
	Fifth Third Bank Financials Diversified Banks 0.06
RF CFG 0.9082653588282964
	Regions Financial Corporation Financials Regional Banks 0.04
	Citizens Financial Group Financials Regional Banks 0.04
UDR ESS 0.9084292508725579
	UDR, Inc. Real Estate Multi-Family Residential REITs 0.03
	Essex Property Trust Real Estate Multi-Family Residential REITs 0.04
ESS EQR 0.9084975133478377
	Essex Property Trust Real Estate Multi-Family Residential REITs 0.04
	Equity Residential Real Estate Multi-Family Residential REITs 0.05
V MA 0.9085857503735966
	Visa Inc. Financials Transaction & Payment Processing Services 0.9
	Mastercard Financials Transaction & Payment Processing Services 0.82
KEY 

In [63]:
sp500Info.loc["AAPL"]

,AAPL
Security,Apple Inc.
GICS Sector,Information Technology
GICS Sub-Industry,"Technology Hardware, Storage & Peripherals"
Headquarters Location,"Cupertino, California"
Date added,1982-11-30
CIK,320193
Founded,1977
